### Imports
* requests: easy HTTP GETs

In [83]:
import requests
from dotenv import load_dotenv
import os

import streamlit as st
import openai
import re
import predictor
import requests
from PIL import Image
from io import BytesIO

#### Get card info
This method retrieves relevant information about cards from Scryfall, which is a database containing Magic: the Gathering cards and the information about them. It stores the information in a map, which maps the information to appropriate names, and returns the map.

In [84]:
def get_card_info(card_name):
        """
        Fetch Magic: The Gathering card information from Scryfall API
        
        Args:
            card_name: Name of the MTG card to search for
            
        Returns:
            Dictionary with card information or None if card not found
        """
        # URL encode the card name for the API request
        url = f"https://api.scryfall.com/cards/named?exact={card_name}"
        
        try:
            response = requests.get(url).json()
            
            # Check if the request was successful
            if response:
                card_data = response
                
                # Extract the requested information
                card_info = {
                    'image_uris': card_data.get('image_uris'),
                    'name': card_data.get('name'),
                    'mana_cost': card_data.get('mana_cost'),
                    'types': card_data.get('type_line'),
                    'oracle_text': card_data.get('oracle_text'),
                    'power': card_data.get('power'),
                    'toughness': card_data.get('toughness'),
                    'loyalty': card_data.get('loyalty')
                }
                
                return card_info
            else:
                print(f"Error: Could not find card named '{card_name}'")
                return None
                
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

In [85]:
system_prompt = """You are a Magic: The Gathering expert specializing in evaluating cards for Standard format play.
    The best standard decks in the foramt are highly value centered and full of removal and powerful threats.
    Analyze the provided card based on its mana cost, types, oracle text, and power/toughness if applicable.
    Rate the card on a scale of 1-5 where:
    
    1: Unplayable in every situation, and outclassed by other cards
    2: Very weak, but potentially playable in niche archetypes or for specific sideboard uses
    3: Strong cards that can be played in a variety of decks, but mainly as a supporting card
    4: Exceptionally strong cards that inspire deck archetypes and provide lots of value on their own
    5: Broken and format-warping card than defines the metagame
    
    Your analysis should include:
    1. Power Rating (1-5)
    2. Strengths of the card
    3. Weaknesses or limitations
    
    Here is an example of a 1: 
    Card Name: "Air Marshal"
    Mana Cost: 1U
    Types: Creature - Human Soldier
    Oracle Text: 3: Target Soldier gains flying until end of turn.
    Power: 2
    Toughness: 1
    
    This card provides a mediocre body for its cost, and the cost of 3 mana to give a creature flying is far too high for the effect.
    It is outclassed by many other cards in the format and does not provide enough value to be worth playing.
    
    Here is an example of a 2:
    Card Name: "Abrade"
    Mana Cost: 1R
    Types: Instant
    Oracle text: Choose one - Abrade deals 3 damage to target creature; or destroy target artifact.
    Power: N/A
    Toughness: N/A
    This card provides decent utility, but at 2 mana it doesn't provide enough value to be worth playing in most decks.
    
    Here is an example of a 3:
    Card Name: Amalia Benavides Aguirre
    Mana Cost: WB
    Types: Legendary Creature - Vampire Scout
    Oracle text: Ward - Pay 3 life. Whenever you gain life, Amalia Benavides Aguirre explores. Then destroy all other creatures if its power is exactly 20. (To have this creature explore, reveal the top card of your library. Put that card into your hand if it is a land. Otherwise, put a +1/+1 counter on this creature, then put the card back or put it into your graveyard.)
    Power: 2
    Toughness: 2
    This card provides excellent value for life gain deck via growing power and toughness and lots of card selection. Becuase of this, it is a powerful addition to decks that gain life incrementally.
    
    Here is an example of a 4: 
    Card Name: "Overlord of the Hauntwoods"
    Mana Cost: 3GG
    Types: Enchantment Creature - Avatar Horror
    Oracle Text: Impending 4—1GreenGreen (If you cast this spell for its impending cost, it enters with four time counters and isn't a creature until the last is removed. At the beginning of your end step, remove a time counter from it.) Whenever this permanent enters or attacks, create a tapped colorless land token named Everywhere that is every basic land type.
    This card provides immense value for ramp decks and domain decks via its ability to create a land token that is every basic land type. Furthermore, it is flexible in that it can be played for full price as a creature, or early for its impending cost.
    This card is very powerful, and finds its way into many decks.
    
    Here is an example of a 5:
    Card Name: "Up the Beanstalk"
    Mana Cost: 1G
    Types: Enchantment
    Oracle text: When Up the Beanstalk enters the battlefield and whenever you cast a spell with mana value 5 or greater, draw a card.
    Power: N/A
    Toughness: N/A
    
    This card provides immediate value upon entering via drawing a card, and provides powerful repeated value throughout the game.
    Furthermore, because of synergies with other strong cards in the format, it is very easy to trigger the card draw effect.
    This card is the reason many of the best decks exist, and is a format-defining card.
    """
    

In [ ]:
API_KEY = "API_KEY"

In [87]:
def analyze_card(card_info):
    try:
        client = openai.OpenAI(api_key=API_KEY)
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": f"Please analyze this Magic: The Gathering card:\n\nName: {card_info['name']}\nMana Cost: {card_info['mana_cost']}\nTypes: {card_info['types']}\nOracle Text: {card_info['oracle_text']}\nPower/Toughness: {card_info['power']}/{card_info['toughness']}\nLoyalty: {card_info['loyalty']}\n\n"
                }
            ],
            temperature=0.7,
            max_tokens=1000
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

In [88]:
card_to_analyze1 = "Arahbo, the First Fang"

info1 = get_card_info(card_to_analyze1)
rating1 = analyze_card(info1)

from IPython.display import display, Markdown
display(Markdown(rating1))

card_to_analyze2 = "Ambush Gigapede"

info2 = get_card_info(card_to_analyze2)
rating2 = analyze_card(info2)

display(Markdown(rating2))


Power Rating: 3

Strengths of the card:
1. Tribal Synergy: Arahbo, the First Fang boosts the power of all other Cat creature types you control, which can be a significant advantage in a Cat tribal deck. This type of global anthem effect (Other Cats you control get +1/+1) is valuable as it can turn even the smallest creatures into more considerable threats.
2. Token Generation: Arahbo has the ability to generate additional value by creating a 1/1 white Cat creature token whenever Arahbo or another nontoken Cat enters the battlefield. This ability helps to maintain and even increase board presence, making it more difficult for the opponent to clear the board with single-target removal or even some mass removal spells.
3. Legendary Creature - Being a legendary creature, Arahbo can be a focal point for decks that capitalize on legendary synergies or commander-oriented strategies within Standard.

Weaknesses or limitations:
1. Specificity: Arahbo’s abilities are very niche, focusing solely on benefiting Cat creatures. This specificity means that its utility is heavily limited to a Cat tribal strategy, reducing its versatility compared to more generally applicable cards.
2. Moderate Impact: While the anthem and token-generation effects are useful, they are not overwhelmingly powerful. The card does not protect itself or other creatures from removal, nor does it offer any immediate game-winning potential. Its 2/2 body for 3 mana is also somewhat fragile and can be easily dealt with by many common removal spells in Standard.
3. Mana Cost: At a cost of {2}{W}, Arahbo is reasonably priced, but in a fast-paced or highly competitive environment, the turn spent casting it might be better utilized for more impactful plays or setting up defenses, especially given its vulnerability.

Overall, Arahbo, the First Fang receives a power rating of 3. It is a strong card in Cat tribal decks, providing both an anthem and a token-generation effect, which can consistently add value over the course of a game. However, its impact is moderate and highly dependent on the presence of other Cats, making it less effective in a broader range of deck types. This card can definitely find a home in specific Standard decks focused around its tribal theme but is unlikely to influence the format outside of that niche.

Power Rating: 2

Strengths of the card:
1. **Flash**: The ability to play Ambush Gigapede at instant speed is a significant strength. It allows the player to surprise an opponent by bringing in a creature during their turn or in response to an action, potentially disrupting their plans.
2. **Removal Effect**: Its entry effect, giving an opponent's creature -2/-2 until end of turn, can serve as a removal or combat trick. This can potentially eliminate a smaller threat or weaken a bigger creature to swing combat in your favor.

Weaknesses or limitations:
1. **High Mana Cost**: At a total of six mana ({4}{B}{B}), Ambush Gigapede is quite costly for the effects it provides. In a format that includes highly efficient and impactful cards, its cost can be a significant drawback.
2. **Fragility**: With a toughness of 2, despite having 6 power, the Gigapede is very susceptible to most forms of removal used in Standard, including cheaper damage-based removal like Shock or Stomp. This fragility undermines its high cost.
3. **Limited Impact**: The -2/-2 effect, while potentially useful, is relatively minor compared to other removal or creature effects available at this or lower mana costs in Standard. It might not be impactful enough against larger threats that commonly appear in the format.

Overall, while Ambush Gigapede has some interesting abilities, its high mana cost combined with its susceptibility to common removal spells and relatively modest impact makes it less desirable in competitive Standard play. It might find a place in more casual builds or specific sideboard scenarios against decks with a lot of x/2 creatures, but it generally doesn't provide enough value or resilience for its cost in a highly competitive environment.